# Creating dynamic flows

In this tutorial, you will:

- Learn how to create dynamic workflows.
- Understand the `detour`, `addition`, and `replace` options in the `Response` object.

The ability create dynamic workflows (i.e. jobs or workflows that launch other jobs or workflows) is
a particularly powerful usage pattern in Jobflow.


## The `Response(replace)` option


The main mechanism for creating dynamic jobs in Jobflow is through the `Response` object. We will demonstrate this below for a toy example where we:

1. Generate a list of numbers whose length is only determined at runtime.
2. Perform a toy operation on each number in the list.

While this is a trivial example, a similar usage is common in computational materials science (e.g. you might perform a calculation on a bulk structure, carve all possible surface slabs, and then perform a calculation on each slab). What makes this dynamic is that the number of jobs is only determined at runtime.


In [23]:
import warnings

warnings.filterwarnings("ignore", "Using `tqdm.autonotebook.tqdm`")

In [24]:
from random import randint
from jobflow import job, Flow, Response
from jobflow.managers.local import run_locally

@job
def make_list(a):
    return [a] * randint(2, 5)

@job
def add(a, b):
    return a + b

@job
def add_distributed(list_a):
    jobs = []
    for val in list_a:
        jobs.append(add(val, 1))
    
    flow = Flow(jobs)
    return Response(replace=flow)

job1 = make_list(2)
job2 = add_distributed(job1.output)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-07 22:51:51,721 INFO Started executing jobs locally
2023-06-07 22:51:51,723 INFO Starting job - make_list (7291e79a-5282-4dcd-b3e6-e64e8f6d66e6)
2023-06-07 22:51:51,725 INFO Finished job - make_list (7291e79a-5282-4dcd-b3e6-e64e8f6d66e6)
2023-06-07 22:51:51,727 INFO Starting job - add_distributed (ab37265d-115b-4322-b8d7-0aebd4fedf7a)
2023-06-07 22:51:51,730 INFO Finished job - add_distributed (ab37265d-115b-4322-b8d7-0aebd4fedf7a)
2023-06-07 22:51:51,733 INFO Starting job - add (c070dd14-10b3-403b-b331-6c1d0878390b)
2023-06-07 22:51:51,734 INFO Finished job - add (c070dd14-10b3-403b-b331-6c1d0878390b)
2023-06-07 22:51:51,736 INFO Starting job - add (317bcdd3-7844-46e0-ab4d-c9064b5e1f12)
2023-06-07 22:51:51,737 INFO Finished job - add (317bcdd3-7844-46e0-ab4d-c9064b5e1f12)
2023-06-07 22:51:51,738 INFO Starting job - add (67bbd11b-869b-466c-883f-f361648ee7a6)
2023-06-07 22:51:51,739 INFO Finished job - add (67bbd11b-869b-466c-883f-f361648ee7a6)
2023-06-07 22:51:51,740 INFO Finis

c:\Users\asros\miniconda3\envs\cms\lib\site-packages\jobflow\utils\graph.py:53: UserWarning: Some jobs are not connected, their ordering may be random
  warnings.warn("Some jobs are not connected, their ordering may be random")


In [25]:
for uuid, response in responses.items():
    print(f"{uuid}: {response}")

7291e79a-5282-4dcd-b3e6-e64e8f6d66e6: {1: Response(output=[2, 2, 2], detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
ab37265d-115b-4322-b8d7-0aebd4fedf7a: {1: Response(output=None, detour=None, addition=None, replace=<jobflow.core.flow.Flow object at 0x000001A3C9E3DF40>, stored_data=None, stop_children=False, stop_jobflow=False)}
c070dd14-10b3-403b-b331-6c1d0878390b: {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
317bcdd3-7844-46e0-ab4d-c9064b5e1f12: {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
67bbd11b-869b-466c-883f-f361648ee7a6: {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


As sene above, there are several jobs that were run --- certainly more than the two we started with. The first job generates a list of 2's with a random length. The second job in the flow is what launches a job on each entry in the list. It is replaced by one job for each entry, hence it has no direct output. Then each newly generated job is run.


## The `Response(addition)` option


Beyond replacing a job with downstream jobs, there is also the option to add jobs to the current flow on-the-fly.

Here we will create a simple flow that:

1. Adds a value to a given number.
2. If the output is less than 10, do the addition again. Otherwise, stop.


In [26]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(addition=add(a, b))
    
job1 = add(1, 2)
job2 = add_with_logic(job1.output, 2)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-07 22:51:51,909 INFO Started executing jobs locally
2023-06-07 22:51:51,911 INFO Starting job - add (2e07f7ce-55ec-449b-8546-4bf3d2a04619)
2023-06-07 22:51:51,915 INFO Finished job - add (2e07f7ce-55ec-449b-8546-4bf3d2a04619)
2023-06-07 22:51:51,919 INFO Starting job - add_with_logic (aa60349c-1ff8-4e20-992f-9722887def37)
2023-06-07 22:51:51,926 INFO Finished job - add_with_logic (aa60349c-1ff8-4e20-992f-9722887def37)
2023-06-07 22:51:51,930 INFO Starting job - add (a0e2c15d-d2c4-41ab-9816-334219f280fa)
2023-06-07 22:51:51,934 INFO Finished job - add (a0e2c15d-d2c4-41ab-9816-334219f280fa)
2023-06-07 22:51:51,936 INFO Finished executing jobs locally


In [27]:
for uuid, response in responses.items():
    print(f"{uuid}: {response}")

2e07f7ce-55ec-449b-8546-4bf3d2a04619: {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
aa60349c-1ff8-4e20-992f-9722887def37: {1: Response(output=None, detour=None, addition=<jobflow.core.flow.Flow object at 0x000001A3C9CD2430>, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
a0e2c15d-d2c4-41ab-9816-334219f280fa: {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


As you can see above, the addition job is correctly run twice. Now let's confirm that the addition job is only run once if the output of the first job is greater than 10.


In [28]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(addition=add(a, b))
    
job1 = add(1, 20)
job2 = add_with_logic(job1.output, 20)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-07 22:51:52,084 INFO Started executing jobs locally
2023-06-07 22:51:52,087 INFO Starting job - add (d16042dd-d603-434e-ad7f-505177fc38aa)
2023-06-07 22:51:52,090 INFO Finished job - add (d16042dd-d603-434e-ad7f-505177fc38aa)
2023-06-07 22:51:52,093 INFO Starting job - add_with_logic (778b8a25-3be9-48b0-ad72-5b5c8e17e656)
2023-06-07 22:51:52,098 INFO Finished job - add_with_logic (778b8a25-3be9-48b0-ad72-5b5c8e17e656)
2023-06-07 22:51:52,100 INFO Finished executing jobs locally


In [29]:
for uuid, response in responses.items():
    print(f"{uuid}: {response}")

d16042dd-d603-434e-ad7f-505177fc38aa: {1: Response(output=21, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
778b8a25-3be9-48b0-ad72-5b5c8e17e656: {1: Response(output=None, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


Now, we see that the `Response(addition)` does not launch a new job.


## The `Response(detour)` option


The `Response(detour)` option behaves similarly to `Response(addition)`. The difference is that `Response(addition)` will add a job (or flow) to the current flow, while `Response(detour)` will no longer run the current flow and will switch to a parallel job or flow.


In [30]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(detour=add(a, b))
    
job1 = add(1, 2)
job2 = add_with_logic(job1.output, 2)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-07 22:51:52,232 INFO Started executing jobs locally
2023-06-07 22:51:52,235 INFO Starting job - add (678ad075-8528-4cec-bcf0-af2da067b87f)
2023-06-07 22:51:52,238 INFO Finished job - add (678ad075-8528-4cec-bcf0-af2da067b87f)
2023-06-07 22:51:52,240 INFO Starting job - add_with_logic (87753486-d4d3-46ab-a3c9-f318b75bf904)
2023-06-07 22:51:52,247 INFO Finished job - add_with_logic (87753486-d4d3-46ab-a3c9-f318b75bf904)
2023-06-07 22:51:52,249 INFO Starting job - add (a6d49e68-13c6-47ad-a688-8eeba9df3cba)
2023-06-07 22:51:52,251 INFO Finished job - add (a6d49e68-13c6-47ad-a688-8eeba9df3cba)
2023-06-07 22:51:52,253 INFO Finished executing jobs locally


In [31]:
for uuid, response in responses.items():
    print(f"{uuid}: {response}")

678ad075-8528-4cec-bcf0-af2da067b87f: {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
87753486-d4d3-46ab-a3c9-f318b75bf904: {1: Response(output=None, detour=<jobflow.core.flow.Flow object at 0x000001A3C9E38C40>, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
a6d49e68-13c6-47ad-a688-8eeba9df3cba: {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


For this toy example, both `Response(addition)` and `Response(detour)` behave identically.
